In [1]:
import numpy as np
import pandas as pd
import sklearn

import lightgbm as lgb
from xgboost import XGBClassifier

from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import statsmodels.api as sm
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore')

%load_ext autotime

time: 185 µs (started: 2022-05-08 17:06:18 +04:00)


In [2]:
filepath = "../Data/TrainDataMean.csv"

train_data = pd.read_csv(filepath)

filepath = "../Data/TestDataMean.csv"

test_data = pd.read_csv(filepath)

time: 3.34 s (started: 2022-05-08 17:06:18 +04:00)


In [3]:
# Dropping "Lead Flag" and "ZIPCode" to run LightGBM model on the train and test datasets

x_train = train_data.drop(['Lead Flag', 'ZIPCode'], axis=1)
y_train = train_data['Lead Flag']

x_test = test_data.drop(['Lead Flag', 'ZIPCode'], axis=1)
y_test = test_data['Lead Flag']

time: 228 ms (started: 2022-05-08 17:06:22 +04:00)


In [4]:
# In this dictionary the results of LightGBM will be added, then dataframe will be made based on this

compare_results_lgbm = {'learning_rate':[], 'max_depth':[], 'score_train':[], 'score_test': []}

time: 487 µs (started: 2022-05-08 17:06:22 +04:00)


In [5]:
# Running nested for loops to put values in the parameters of LightGBM, run the model, save the train and test scores. 

results = []
for i in np.arange (0.1, 0.9,0.1):
    for j in range(-5,5, 2):
        model = lgb.LGBMClassifier(learning_rate=i, max_depth=j, random_state=42)
        model.fit(x_train, y_train, eval_set=[(x_train,y_train)], eval_metric='AUC', verbose=20)
        
        compare_results_lgbm['learning_rate'].append(i)
        compare_results_lgbm['max_depth'].append(j)
        
        Y0_lgbm=model.predict_proba(x_train)[:,1]
        Y1_lgbm=model.predict_proba(x_test)[:,1]
        
        compare_results_lgbm['score_train'].append(roc_auc_score(y_train, Y0_lgbm))
        compare_results_lgbm['score_test'].append(roc_auc_score(y_test, Y1_lgbm))

[20]	training's auc: 0.734391	training's binary_logloss: 0.0496352
[40]	training's auc: 0.770554	training's binary_logloss: 0.0476445
[60]	training's auc: 0.797229	training's binary_logloss: 0.0460777
[80]	training's auc: 0.820466	training's binary_logloss: 0.0446828
[100]	training's auc: 0.83952	training's binary_logloss: 0.0433495
[20]	training's auc: 0.734391	training's binary_logloss: 0.0496352
[40]	training's auc: 0.770554	training's binary_logloss: 0.0476445
[60]	training's auc: 0.797229	training's binary_logloss: 0.0460777
[80]	training's auc: 0.820466	training's binary_logloss: 0.0446828
[100]	training's auc: 0.83952	training's binary_logloss: 0.0433495
[20]	training's auc: 0.734391	training's binary_logloss: 0.0496352
[40]	training's auc: 0.770554	training's binary_logloss: 0.0476445
[60]	training's auc: 0.797229	training's binary_logloss: 0.0460777
[80]	training's auc: 0.820466	training's binary_logloss: 0.0446828
[100]	training's auc: 0.83952	training's binary_logloss: 0.043

[80]	training's auc: 0.751798	training's binary_logloss: 0.0496698
[100]	training's auc: 0.757527	training's binary_logloss: 0.0493517
[20]	training's auc: 0.712358	training's binary_logloss: 0.0609553
[40]	training's auc: 0.710914	training's binary_logloss: 0.0653626
[60]	training's auc: 0.708372	training's binary_logloss: 0.0664007
[80]	training's auc: 0.710591	training's binary_logloss: 0.068065
[100]	training's auc: 0.708599	training's binary_logloss: 0.0739059
[20]	training's auc: 0.712358	training's binary_logloss: 0.0609553
[40]	training's auc: 0.710914	training's binary_logloss: 0.0653626
[60]	training's auc: 0.708372	training's binary_logloss: 0.0664007
[80]	training's auc: 0.710591	training's binary_logloss: 0.068065
[100]	training's auc: 0.708599	training's binary_logloss: 0.0739059
[20]	training's auc: 0.712358	training's binary_logloss: 0.0609553
[40]	training's auc: 0.710914	training's binary_logloss: 0.0653626
[60]	training's auc: 0.708372	training's binary_logloss: 0.06

In [6]:
# Making dataframe from dictionary. Getting the best test score for LightGBM and corresponding parameters.

compare_results_lgbm = pd.DataFrame.from_dict(compare_results_lgbm)
compare_results_lgbm[compare_results_lgbm['score_test'] == max(compare_results_lgbm['score_test'])] #0.2 -1

,learning_rate,max_depth,score_train,score_test
28,0.6,1,0.717536,0.713738


time: 26.7 ms (started: 2022-05-08 17:12:25 +04:00)


In [7]:
# In this dictionary the results of XGBoost will be added, then dataframe will be made based on this

compare_results_xgb = {'learning_rate':[], 'max_depth':[], 'score_train':[], 'score_test': []}

time: 204 µs (started: 2022-05-08 17:12:25 +04:00)


In [8]:
# Running nested for loops to put values in the parameters of XGBoost, run the model, save the train and test scores. 

results = []
for i in np.arange (0.1, 0.9,0.2):
    for j in range(1,10, 4):
        print('learning rate: ', i, " max_depth: ", j)
        model = XGBClassifier(learning_rate=i, max_depth=j, random_state=42)
        model.fit(x_train, y_train, eval_set=[(x_train, y_train)], eval_metric='auc', verbose=20)
        
        compare_results_xgb['learning_rate'].append(i)
        compare_results_xgb['max_depth'].append(j)
        
        
        Y0_xgb=model.predict_proba(x_train)[:,1]
        Y1_xgb=model.predict_proba(x_test)[:,1]
        
        compare_results_xgb['score_train'].append(roc_auc_score(y_train, Y0_xgb))
        compare_results_xgb['score_test'].append(roc_auc_score(y_test, Y1_xgb))

learning rate:  0.1  max_depth:  1
[0]	validation_0-auc:0.59582
[20]	validation_0-auc:0.59582
[40]	validation_0-auc:0.65484
[60]	validation_0-auc:0.67245
[80]	validation_0-auc:0.68376
[99]	validation_0-auc:0.69037
learning rate:  0.1  max_depth:  5
[0]	validation_0-auc:0.63420
[20]	validation_0-auc:0.69313
[40]	validation_0-auc:0.71731
[60]	validation_0-auc:0.74685
[80]	validation_0-auc:0.76709
[99]	validation_0-auc:0.77776
learning rate:  0.1  max_depth:  9
[0]	validation_0-auc:0.63526
[20]	validation_0-auc:0.73398
[40]	validation_0-auc:0.83351
[60]	validation_0-auc:0.90051
[80]	validation_0-auc:0.92198
[99]	validation_0-auc:0.93354
learning rate:  0.30000000000000004  max_depth:  1
[0]	validation_0-auc:0.59582
[20]	validation_0-auc:0.67233
[40]	validation_0-auc:0.69548
[60]	validation_0-auc:0.70466
[80]	validation_0-auc:0.70960
[99]	validation_0-auc:0.71271
learning rate:  0.30000000000000004  max_depth:  5
[0]	validation_0-auc:0.63420
[20]	validation_0-auc:0.74425
[40]	validation_0-

In [9]:
# Making dataframe from dictionary. Getting the best test score for XGBoost and corresponding parameters.

compare_results_xgb = pd.DataFrame.from_dict(compare_results_xgb)
compare_results_xgb[compare_results_xgb['score_test'] == max(compare_results_xgb['score_test'])] #0.2 -1

,learning_rate,max_depth,score_train,score_test
9,0.7,1,0.718745,0.713971


time: 16.6 ms (started: 2022-05-08 17:28:16 +04:00)


In [10]:
# Running Logistic Regression without regularization 

logreg = LogisticRegression(random_state=42)

logreg.fit(x_train,y_train)

Y0_logreg=logreg.predict_proba(x_train)[:,1]
Y1_logreg=logreg.predict_proba(x_test)[:,1]

print("ROC_AUC Train:", roc_auc_score (y_train, Y0_logreg))
print ("ROC_AUC Test:", roc_auc_score(y_test, Y1_logreg))

ROC_AUC Train: 0.5316997123894613
ROC_AUC Test: 0.5266451716890503
time: 12.2 s (started: 2022-05-08 17:28:16 +04:00)


In [11]:
# Running Logistic Regression with regularization. Better results than without regularization

logreg=LogisticRegression(C=1,penalty='l1',solver="liblinear", random_state=42)

logreg.fit(x_train,y_train)

Y0_logreg=logreg.predict_proba(x_train)[:,1]
Y1_logreg=logreg.predict_proba(x_test)[:,1]

print("ROC_AUC Train:", roc_auc_score (y_train, Y0_logreg))
print ("ROC_AUC Test:", roc_auc_score(y_test, Y1_logreg))

ROC_AUC Train: 0.7046583518729963
ROC_AUC Test: 0.7076435614856322
time: 1min 9s (started: 2022-05-08 17:28:29 +04:00)
